<a href="https://colab.research.google.com/github/oigwe-frx/data-sourcing-challenge/blob/main/osita_retrieve_movie_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Required Libraries and Set Up Environment Variables

In [60]:
# Dependencies
import requests
import time
#from dotenv import load_dotenv
import os
import pandas as pd
import json
from urllib.parse import urlencode
import re

In [61]:
# Set environment variables from the .env in the local environment
from google.colab import userdata
nyt_api_key = userdata.get('NYT_API_KEY')

tmdb_api_key=userdata.get('TMDB_API_KEY')

tmdb_access_token=userdata.get('TMDB_ACCESS_KEY')

### Access the New York Times API

In [62]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
params = {
  'api-key': nyt_api_key,
  'fq': filter_query,
  'sort': sort,
  'fl': field_list,
  'begin_date': begin_date,
  'end_date': end_date,
  }

# Construct the URL with properly encoded parameters
encoded_params = urlencode(params)
nyt_query_url = url + encoded_params

In [63]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_params = {
        'api-key': userdata.get('NYT_API_KEY'),
        'fq': filter_query,
        'sort': sort,
        'fl': field_list,
        'begin_date': begin_date,
        'end_date': end_date,
        'page': page
    }

    # Make a "GET" request and retrieve the JSON
    response = requests.get(url, params=query_params)
    data = response.json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)


    #Try and save the reviews to the reviews_list
    if data["response"]["docs"]:

      # loop through the reviews["response"]["docs"] and append each review to the list
      for review in data["response"]["docs"]:
        reviews_list.append(review)

      # Print the page that was just retrieved
      print(f"Checked page {page}")

    else:
      # Print the page number that had no results then break from the loop
      print(f"No results on page {page}, stopping.")
      break



Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [64]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

json_data = json.dumps(reviews_list[:5], indent=4)
json_data


'[\n    {\n        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",\n        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",\n        "source": "The New York Times",\n        "headline": {\n            "main": "\\u2018The Attachment Diaries\\u2019 Review: Love, Sick",\n            "kicker": null,\n            "content_kicker": null,\n            "print_headline": "The Attachment Diaries",\n            "name": null,\n            "seo": null,\n            "sub": null\n        },\n        "keywords": [\n            {\n                "name": "subject",\n                "value": "Movies",\n                "rank": 1,\n                "major": "N"\n            },\n            {\n                "name": "creative_works",\n                "value": "The Attachment Diaries (Movie)",\n                "rank": 2,\n                "major": "N"\n            },\n            {\n     

In [65]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

# Convert reviews_list to a Pandas DataFrame
reviews_df = pd.json_normalize(reviews_list)

# Print the DataFrame
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [66]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early


def extract_title(row):
    # Search for text between Unicode quotes, and exclude any trailing " Review" if present
    match = re.search(r'\u2018(.*?)\u2019(?: Review)?', row)
    if match:
        # Return just the captured title part
        return match.group(1)
    return None  # Return None if no match is found

# Create a new column "title" by applying the extract_title function to "headline.main"
reviews_df["title"] = reviews_df["headline.main"].apply(extract_title)

# Print the DataFrame with the new column
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [67]:
reviews_df['keywords'][0]

def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Using the function
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [68]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()
titles

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love 

### Access The Movie Database API

In [69]:
# Function to Extract Genre
def extract_genres(genre_list):
    extracted_genres = []
    for item in genre_list:
        # Extract 'name' and append the keyword item to the extracted_genres list
        extracted_genres.append(f"{item['name']}")
    return extracted_genres

# Function to Extract English Name of Movie
def extract_english_name(language_list):
    extracted_genres = []
    for item in language_list:
        # Extract the spoken_languages' English name into a list
        if item['english_name']:
          extracted_genres.append(f"{item['english_name']}")
    return extracted_genres

# Function to Extract Production Country of Movie
def extract_production_country(country_list):
    extracted_countries = []
    for item in country_list:
        # Extract the country_list' into a list
        extracted_countries.append(f"{item['name']}")
    return extracted_countries

In [70]:

# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="

In [71]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in titles:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(30)

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    headers = {
      "accept": "application/json",
      "Authorization": f"Bearer {tmdb_access_token}"
    }

    response_general_query = requests.get(
      f"https://api.themoviedb.org/3/search/movie?query={title}",
      headers=headers
    )
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
        # Get movie id
        response_movies_id = response_general_query.json()["results"][0]["id"]

        # Make a request for a the full movie details
        # Execute "GET" request with url
        response_full_details = requests.get(
          f"https://api.themoviedb.org/3/movie/{response_movies_id}",
          headers=headers
        )
        print(response_full_details.json())
        # Extract the genre names into a list
        response_genres = extract_genres(response_full_details.json()["genres"])

        # Extract the spoken_languages' English name into a list
        response_spoken_languages = extract_english_name(response_full_details.json()["spoken_languages"])

        # Extract the production_countries' name into a list
        response_production_countries = extract_production_country(response_full_details.json()["production_countries"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            **response_full_details.json(),
            "genres": response_genres,
            "spoken_languages": response_spoken_languages,
            "production_countries": response_production_countries

        })

        # Print out the title that was found
        print(f"Found {title}")
    except IndexError:
        print(f"{title} not found.")


{'adult': False, 'backdrop_path': None, 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}], 'homepage': '', 'id': 743040, 'imdb_id': 'tt11359158', 'origin_country': ['ES'], 'original_language': 'es', 'original_title': 'El apego', 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'popularity': 5.486, 'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg', 'production_companies': [], 'production_countries': [{'iso_3166_1': 'AR', 'name': 'Argentina'}], 'release_date': '2021-10-07', 'revenue': 0, 'runtime': 102, 'spoken_la

In [72]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# Convert each dictionary in the list to a JSON string
tmdb_movies_list_strings = [json.dumps(movie, indent=4) for movie in tmdb_movies_list[:5]]

# Print the results to see the JSON strings
for movie_str in tmdb_movies_list_strings:
    print(movie_str)

{
    "adult": false,
    "backdrop_path": null,
    "belongs_to_collection": null,
    "budget": 0,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "homepage": "",
    "id": 743040,
    "imdb_id": "tt11359158",
    "origin_country": [
        "ES"
    ],
    "original_language": "es",
    "original_title": "El apego",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 5.486,
    "poster_path": "/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg",
    "production_companies": [],
    "production_countries": [
        "Argentina"
    ],
    "release_date": "2021-10-07",
    "revenue": 0,

In [73]:
# Convert the results to a DataFrame
tmdb_search_results_df = pd.json_normalize(tmdb_movies_list)
tmdb_search_results_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,status,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path
0,False,None,NaN,0,"[Drama, Mystery, Thriller, Horror]",,743040,tt11359158,[ES],es,...,Released,,The Attachment Diaries,False,3.000,4,NaN,NaN,NaN,NaN
1,False,None,NaN,0,"[Comedy, Drama]",,1242065,None,[AU],en,...,Released,What in... What the... What...?,What...,False,0.000,0,NaN,NaN,NaN,NaN
2,False,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,NaN,0,"[Drama, Romance]",https://gooddeedentertainment.com/you-can-live...,887580,tt15782570,[CA],en,...,Released,,You Can Live Forever,False,6.700,33,NaN,NaN,NaN,NaN
3,False,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,NaN,0,"[Romance, Comedy]",https://www.netflix.com/title/81424906,813726,tt20115096,[US],en,...,Released,,A Tourist's Guide to Love,False,6.300,158,NaN,NaN,NaN,NaN
4,False,/fDi3FeuOZlI8yzCu3RKvIsgRnct.jpg,NaN,0,"[Comedy, Drama]",,371449,tt4158876,[US],en,...,Released,,Other People,False,6.300,193,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,False,/eTovTpBgekkg9HZ5wGXUiiWys9M.jpg,NaN,0,"[Drama, Romance]",http://theotherhalfmovie.com/,381015,tt4625334,[CA],en,...,Released,Yet memory will not abandon love.,The Other Half,False,6.300,24,NaN,NaN,NaN,NaN
195,False,/5kmoreEiqaU0JoE8GZmLwLvAsJq.jpg,NaN,40000000,"[Romance, Drama, War]",https://www.uphe.com/movies/the-ottoman-lieute...,433356,tt4943322,[US],en,...,Released,In a land on the brink of war the most dangero...,The Ottoman Lieutenant,False,6.173,246,NaN,NaN,NaN,NaN
196,False,/ugnzz1A9mjJUgnFwVqEUsUNt7t6.jpg,NaN,0,"[Romance, Comedy]",,1005681,tt20198716,[ES],es,...,Released,,"Love, Divided",False,6.687,91,NaN,NaN,NaN,NaN
197,False,/aAeUP99GEhJr2KIHzfA9chqqXUD.jpg,NaN,2000000,"[Romance, Comedy]",,431093,tt5537228,"[US, MX]",en,...,Released,"If her heart doesn't choose, her family will.",Everybody Loves Somebody,False,6.600,158,NaN,NaN,NaN,NaN


### Merge and Clean the Data for Export

In [74]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_search_results_df, on="title", how="inner")
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,spoken_languages,status,tagline,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,[Spanish],Released,,False,3.000,4,NaN,NaN,NaN,NaN
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"[English, French]",Released,,False,6.700,33,NaN,NaN,NaN,NaN
2,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,[English],Released,,False,6.300,193,NaN,NaN,NaN,NaN
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,"[English, Spanish]",Released,First love meets new love.,False,6.406,69,NaN,NaN,NaN,NaN
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,[English],Released,John Lennon and May Pang shared a weekend that...,False,6.000,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",...,[English],Released,Risk everything... for love.,False,7.247,3632,NaN,NaN,NaN,NaN
142,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,[English],Released,Have you seen this girl?,False,6.100,326,NaN,NaN,NaN,NaN
143,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,"[Armenian, German, English, French]",Released,Empires fall. Love survives.,False,6.900,449,NaN,NaN,NaN,NaN
144,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,...,[English],Released,Yet memory will not abandon love.,False,6.300,24,NaN,NaN,NaN,NaN


In [75]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [76]:
# Drop "byline.person" column


In [77]:
# Delete duplicate rows and reset index


In [78]:
# Export data to CSV without the index
